In [1]:
# imports
import pandas as pd
import numpy as np
import itertools as it

import matplotlib.pyplot as plt

import time
import datetime
import os
import shutil

import warnings
warnings.filterwarnings("ignore")

In [2]:
# only colab environment
#from google.colab import drive  
#drive.mount('/content/drive')
# v_path = '/content/drive/MyDrive/RL_DIC/BENCHMARK/' 

v_path = '' # local environment

In [3]:
# DATA

def get_ohlcv_data(market, timeframe, init_date, end_date):
    """
    Genera df ohlcv para un determinado mercado y un determinado timeframe y periodo de tiempo
    """
    
    data = pd.read_pickle(v_path + market + '.pkl') # leemos el pkl del activo
    data['date'] = pd.to_datetime(data['Date']+ ' ' + data['Time'], format="%m/%d/%Y %H:%M") # definimos la columna date con Date y Time y la ponemos como indice
    data = data.set_index('date')
    data = data.drop(['Time'], axis=1)
    data = data.iloc[:,[1,2,3,4,5,6]]  # seleccionamos las columnas que queremos
    data.columns = ['open','high','low','close','up','down']  # renombramos las columnas
    data = data.loc[init_date:end_date] # seleccionamos el el rango de fechas que queremos

    # hacemos "resample" al timeframe seleccionado
    data = data.resample(timeframe).agg({'open':'first','high':'max','low':'min','close':'last','up':'sum','down':'sum'})
    df_data = data.dropna()  # eliminamos los NaN
    return df_data

In [4]:
# INDICATORS

def df_indicators_gen(df_data):
    """
    Esta funcion recibe un df con los datos ohlcv para un determinado activo-timeframe-periodo
    Se definen diferentes indicadores y patrones (de momentum, medias moviles, canales, breakout, volatilidad...), cada uno de ellos con parametros fijos.
    Los calculos son vectorizados
    La salida de la funcion es el df con los indicadores anadidos como columnas
    """
    df = df_data.copy()

    # ema/sma (6, 12, 26, 60, 120, 200)
    df['ema_6'] = df['close'].ewm(span=6, min_periods=1, adjust=False).mean()
    df['ema_12'] = df['close'].ewm(span=12, min_periods=1, adjust=False).mean()
    df['ema_26'] = df['close'].ewm(span=26, min_periods=1, adjust=False).mean()
    df['sma_60'] = df['close'].rolling(window=60, min_periods=1).mean()
    df['sma_120'] = df['close'].rolling(window=120, min_periods=1).mean()
    df['sma_200'] = df['close'].rolling(window=200, min_periods=1).mean()

    # atr (14, + atr diario de 5min, 10 min, 15mi y 1h)
    df['atr_14'] = np.max(pd.concat([df['high'] - df['low'], np.abs(df['high'] - df['close'].shift()), np.abs(df['low'] - df['close'].shift())], axis=1), axis=1).rolling(14).sum()/14
    df['atr_23'] = np.max(pd.concat([df['high'] - df['low'], np.abs(df['high'] - df['close'].shift()), np.abs(df['low'] - df['close'].shift())], axis=1), axis=1).rolling(23).sum()/23
    df['atr_92'] = np.max(pd.concat([df['high'] - df['low'], np.abs(df['high'] - df['close'].shift()), np.abs(df['low'] - df['close'].shift())], axis=1), axis=1).rolling(92).sum()/92
    df['atr_138'] = np.max(pd.concat([df['high'] - df['low'], np.abs(df['high'] - df['close'].shift()), np.abs(df['low'] - df['close'].shift())], axis=1), axis=1).rolling(138).sum()/138
    df['atr_276'] = np.max(pd.concat([df['high'] - df['low'], np.abs(df['high'] - df['close'].shift()), np.abs(df['low'] - df['close'].shift())], axis=1), axis=1).rolling(276).sum()/276

    # minimo/maximo (6, 14, 200)
    df['min_6'] = df['close'].rolling(window=6, min_periods=1).min()
    df['max_6'] = df['close'].rolling(window=6, min_periods=1).max()
    df['min_14'] = df['close'].rolling(window=14, min_periods=1).min()
    df['max_14'] = df['close'].rolling(window=14, min_periods=1).max()
    df['min_200'] = df['close'].rolling(window=200, min_periods=1).min()
    df['max_200'] = df['close'].rolling(window=200, min_periods=1).max()

    # n_bars_up/down (2, 3)
    df['2_bars_up'] = (df['close'] > df['open']) & (df['close'].shift() > df['open'].shift())
    df['2_bars_down'] = (df['close'] < df['open']) & (df['close'].shift() < df['open'].shift())
    df['3_bars_up'] = (df['close'] > df['open']) & (df['close'].shift() > df['open'].shift()) & (df['close'].shift(2) > df['open'].shift(2))
    df['3_bars_down'] = (df['close'] < df['open']) & (df['close'].shift() < df['open'].shift()) & (df['close'].shift(2) < df['open'].shift(2))

    # bollinger_bands (20-2)
    df['bb_upband_20__2'] = df['close'].rolling(20).mean() + (df['close'].rolling(20).std() * 2)
    df['bb_lowband_20__2'] = df['close'].rolling(20).mean() - (df['close'].rolling(20).std() * 2)

    df_indicators = df.dropna() # quitamos las primeras filas, en las que no ha habido suficientes barras para el calculo de los indicadores
    return df_indicators

In [5]:
# CONDITIONS

def df_conditions_gen(df_indicators):
    """
    Esta funcion recibe el df con los indicadores generados anteriormente, con ello definiremos una serie de condiciones o reglas que se usaran para construir los sistemas de trading.
    Se definen 2 sets de condiciones que representan si estamos por encima o por debajo de un canal de precios variable segun el indicador
    Segun el tipo de sistema (tendencial o reversion), como veremos, las condiciones seran entradas o salidas a mercado
    Los dataframes generados tendran como columnas las condiciones, que seran True o False, segun se cumplan o no en cada barra
    """

    df_above_conditions = df_indicators.iloc[:, [0,1,2,3]] 
    df_below_conditions = df_indicators.iloc[:, [0,1,2,3]]

    # inicialmente definimos las condiciones con respecto al cierre de la barra: close
    # probar lo mismo con respecto al "typical value", que es mas representativo de la barra que el close: typical_value= (high+low+close)/3 (o weighted= (high+low+2*close)/4)

    # definimos condiciones simples:
    df_above_conditions['a_condition_1'] = df_indicators['3_bars_up'] == True
    df_above_conditions['a_condition_2'] = df_indicators['2_bars_up'] == True
    df_below_conditions['b_condition_1'] = df_indicators['3_bars_down'] == True
    df_below_conditions['b_condition_2'] = df_indicators['2_bars_down'] == True

    df_above_conditions['a_condition_3'] = df_indicators['close'] > df_indicators['bb_upband_20__2']
    df_below_conditions['b_condition_3'] = df_indicators['close'] < df_indicators['bb_lowband_20__2']

    df_above_conditions['a_condition_4'] = df_indicators['close'] > df_indicators['max_6']
    df_below_conditions['b_condition_4'] = df_indicators['close'] < df_indicators['min_6']
    df_above_conditions['a_condition_5'] = df_indicators['close'] > df_indicators['max_14']
    df_below_conditions['b_condition_5'] = df_indicators['close'] < df_indicators['min_14']
    df_above_conditions['a_condition_6'] = df_indicators['close'] > df_indicators['max_200']
    df_below_conditions['b_condition_6'] = df_indicators['close'] < df_indicators['min_200']

    df_above_conditions['a_condition_7'] = df_indicators['close'] > df_indicators['ema_6']
    df_below_conditions['b_condition_7'] = df_indicators['close'] < df_indicators['ema_6']
    df_above_conditions['a_condition_8'] = df_indicators['close'] > df_indicators['ema_12']
    df_below_conditions['b_condition_8'] = df_indicators['close'] < df_indicators['ema_12']
    df_above_conditions['a_condition_9'] = df_indicators['close'] > df_indicators['ema_26']
    df_below_conditions['b_condition_9'] = df_indicators['close'] < df_indicators['ema_26']

    df_above_conditions['a_condition_10'] = df_indicators['close'] > df_indicators['sma_60']
    df_below_conditions['b_condition_10'] = df_indicators['close'] < df_indicators['sma_60']
    df_above_conditions['a_condition_11'] = df_indicators['close'] > df_indicators['sma_120']
    df_below_conditions['b_condition_11'] = df_indicators['close'] < df_indicators['sma_120']
    df_above_conditions['a_condition_12'] = df_indicators['close'] > df_indicators['sma_200']
    df_below_conditions['b_condition_12'] = df_indicators['close'] < df_indicators['sma_200']

    # podemos tambien definir condiciones combinadas o mas complejas, por ejemplo: 
    df_above_conditions['a_condition_13'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 1.4 * df_indicators['atr_14']) 
    df_above_conditions['a_condition_14'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2 * df_indicators['atr_14'])
    df_above_conditions['a_condition_15'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 1.4 * df_indicators['atr_23'])
    df_above_conditions['a_condition_16'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2 * df_indicators['atr_23'])
    df_above_conditions['a_condition_17'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 1.4 * df_indicators['atr_92'])
    df_above_conditions['a_condition_18'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2 * df_indicators['atr_92'])
    df_above_conditions['a_condition_19'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 1.4 * df_indicators['atr_138'])
    df_above_conditions['a_condition_20'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2 * df_indicators['atr_138'])
    df_above_conditions['a_condition_21'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 1.4 * df_indicators['atr_276'])
    df_above_conditions['a_condition_22'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2 * df_indicators['atr_276'])

    df_below_conditions['b_condition_13'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 1.4 * df_indicators['atr_14'])
    df_below_conditions['b_condition_14'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2 * df_indicators['atr_14'])
    df_below_conditions['b_condition_15'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 1.4 * df_indicators['atr_23'])
    df_below_conditions['b_condition_16'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2 * df_indicators['atr_23'])
    df_below_conditions['b_condition_17'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 1.4 * df_indicators['atr_92'])
    df_below_conditions['b_condition_18'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2 * df_indicators['atr_92'])
    df_below_conditions['b_condition_19'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 1.4 * df_indicators['atr_138'])
    df_below_conditions['b_condition_20'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2 * df_indicators['atr_138'])
    df_below_conditions['b_condition_21'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 1.4 * df_indicators['atr_276'])
    df_below_conditions['b_condition_22'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2 * df_indicators['atr_276'])


    df_above_conditions['a_condition_23'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2.8 * df_indicators['atr_14'])
    df_above_conditions['a_condition_24'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2.8 * df_indicators['atr_23'])
    df_above_conditions['a_condition_25'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2.8 * df_indicators['atr_92'])
    df_above_conditions['a_condition_26'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2.8 * df_indicators['atr_138'])
    df_above_conditions['a_condition_27'] = df_indicators['close'] > (df_indicators['close'].shift(2) + 2.8 * df_indicators['atr_276'])

    df_below_conditions['b_condition_23'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2.8 * df_indicators['atr_14'])
    df_below_conditions['b_condition_24'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2.8 * df_indicators['atr_23'])
    df_below_conditions['b_condition_25'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2.8 * df_indicators['atr_92'])
    df_below_conditions['b_condition_26'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2.8 * df_indicators['atr_138'])
    df_below_conditions['b_condition_27'] = df_indicators['close'] < (df_indicators['close'].shift(2) - 2.8 * df_indicators['atr_276'])

    return df_above_conditions, df_below_conditions    

In [6]:
# STRATEGIES GENERATOR

def df_strategies_gen(df_above_conditions, df_below_conditions, timeframe):
    """
    Esta funcion genera estrategias por combinacion de las reglas definidas anteriormente. Se definen las estrategias con 2 reglas de entrada y 2 de salida, 
    que seran aplicadas a cada activo y timeframe.
    Inicialmente solo sistemas tendenciales, (largos y cortos)
    """

    n_in_rules = n_out_rules = 2 
    conditions_combinations = list(set(list(it.product(list(it.combinations(df_above_conditions.columns[4:], n_in_rules)), 
                                list(it.combinations(df_below_conditions.columns[4:], n_out_rules))))))

    df_strategies = pd.DataFrame(columns=['market', 'timeframe', 'system_type', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2'])

    system_type = 'trend_long'
    for i, combination in enumerate(conditions_combinations):
        df_strategies = df_strategies.append({'market':market, 'timeframe':timeframe, 'system_type':system_type,
                                                'in_condition_1':combination[0][0], 'in_condition_2':combination[0][1],
                                                'out_condition_1':combination[1][0], 'out_condition_2':combination[1][1]}, ignore_index=True)

    system_type = 'trend_short'
    for i, combination in enumerate(conditions_combinations):
        df_strategies = df_strategies.append({'market':market, 'timeframe':timeframe, 'system_type':system_type,
                                                'in_condition_1':combination[1][0], 'in_condition_2':combination[1][1],
                                                'out_condition_1':combination[0][0], 'out_condition_2':combination[0][1]}, ignore_index=True)

    return df_strategies

In [7]:
# TRADES & RETURNS

def generate_trades(system_type, df_bkt, point_value):
    """
    Esta funcion genera las senales y posiciones de la estrategia segun el tipo de sistema, y calcula los trades generados en $
    Se generara señal de entrada cuando se den las 2 condiciones de entrada, y señal de salida cuando se dé al menos una condicion de salida
    """

    # anadimos las senales y posiciones de la estrategia segun el tipo de sistema
    if system_type == 'trend_long':
        df_bkt['signal'] = np.where((df_bkt['in_condition_1'] & df_bkt['in_condition_2']), 1, np.where((df_bkt['out_condition_1'] | df_bkt['out_condition_2']), 0, np.nan))

        df_bkt['signal'].ffill(inplace=True)     # propagamos la senal
        df_bkt['signal'].fillna(0, inplace=True)
        df_bkt['position'] = df_bkt.signal.shift(1) # la posicion se dara en la barra siguiente a la senal

    # calculamos los retornos respecto al open
        df_bkt['dif_open'] = df_bkt.open.pct_change()
        df_bkt['dif_close'] = df_bkt.close.pct_change()
        df_bkt['returns'] = df_bkt.position.shift(1) * df_bkt.dif_open

    # andimos los trades generados por la estrategia
        df_bkt['trade_price'] = np.where((df_bkt.position.shift()== 0) & (df_bkt.position== 1), df_bkt.open, np.where((df_bkt.position.shift() == 1) & (df_bkt.position == 0), df_bkt.open, np.NaN))  
        df_bkt['trade_price'] = df_bkt.trade_price.fillna(method='ffill')
        df_bkt['trade_price'] = np.where((df_bkt.position.shift()== 0) & (df_bkt.position== 0), 0, df_bkt.trade_price)
        df_bkt['trades_closed'] = np.where((df_bkt.position.shift() == 1) & (df_bkt.position == 0), (df_bkt.trade_price - df_bkt.trade_price.shift()), 0)

    # multiplicamos la columna trades_closed por el point value del activo para obtener los valores en $
        df_bkt['trades_closed'] = df_bkt['trades_closed'] * point_value 


    elif system_type == 'trend_short':
        df_bkt['signal'] = np.where((df_bkt['in_condition_1'] & df_bkt['in_condition_2']), -1, np.where((df_bkt['out_condition_1'] | df_bkt['out_condition_2']), 0, np.nan))

        df_bkt['signal'].ffill(inplace=True)     # propagamos la senal
        df_bkt['signal'].fillna(0, inplace=True)
        df_bkt['position'] = df_bkt.signal.shift(1) # la posicion se dara en la barra siguiente a la senal

    # calculamos los retornos respecto al open
        df_bkt['dif_open'] = df_bkt.open.pct_change()
        df_bkt['dif_close'] = df_bkt.close.pct_change()
        df_bkt['returns'] = df_bkt.position.shift(1) * df_bkt.dif_open
        df_bkt['benchmarks_returns'] = df_bkt.dif_close

    # andimos los trades generados por la estrategia
        df_bkt['trade_price'] = np.where((df_bkt.position.shift()== 0) & (df_bkt.position== -1), df_bkt.open, np.where((df_bkt.position.shift() == -1) & (df_bkt.position == 0), df_bkt.open, np.NaN))
        df_bkt['trade_price'] = df_bkt.trade_price.fillna(method='ffill')
        df_bkt['trade_price'] = np.where((df_bkt.position.shift()== 0) & (df_bkt.position== 0), 0, df_bkt.trade_price)
        df_bkt['trades_closed'] = np.where((df_bkt.position.shift() == -1) & (df_bkt.position == 0), (df_bkt.trade_price.shift() - df_bkt.trade_price), 0)

    # multiplicamos la columna trades_closed por el point value del activo para obtener los valores en $
        df_bkt['trades_closed'] = df_bkt['trades_closed'] * point_value 
        
    df_bkt_trades = df_bkt
    df_bkt_trades = df_bkt_trades.dropna()

    return df_bkt_trades

In [8]:
# INITIAL BACKTEST

def backtest(market, point_value, timeframe, system_type, in_condition_1, in_condition_2, out_condition_1, out_condition_2, df_conditions, min_n_trades):
    """
    Esta funcion realiza un backtest basico de cada estrategia generada para eliminar aquellas que no realizan un minimo numero de trades
    """

    df_ratios = pd.DataFrame(columns=['market', 'timeframe', 'system_type', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2', 'n_trades', 'profit_factor', 'bmo', 'net_profit'])
    df_bkt = df_conditions[['open', 'high', 'low', 'close', in_condition_1, in_condition_2, out_condition_1, out_condition_2]]
    df_bkt.columns = ['open', 'high', 'low', 'close', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2']
    df_bkt_trades = generate_trades(system_type, df_bkt, point_value)

    # en primer lugar contamos el numero de trades, ya que si es menor que min_n_trades, no se considera la estrategia
    n_trades = len(df_bkt_trades[df_bkt_trades.trades_closed != 0])
    #print('numero de trades: ', n_trades)
    if n_trades >= min_n_trades:
        # realizamos un primer backtest para obtener el profit factor y el bmo
        profit_factor = df_bkt_trades.trades_closed[df_bkt_trades.trades_closed > 0].sum() / abs(df_bkt_trades.trades_closed[df_bkt_trades.trades_closed < 0].sum())
        bmo = df_bkt_trades.trades_closed[df_bkt_trades.trades_closed != 0].sum() / n_trades
        net_profit = df_bkt_trades.trades_closed.sum()
        
        df_ratios.loc[0] = [market, timeframe, system_type, in_condition_1, in_condition_2, out_condition_1, out_condition_2, n_trades, profit_factor, bmo, net_profit]

    return df_ratios

In [9]:
# MAIN

def main_1(market, timeframes, init_date, end_date):

    # 1. generacion de estrategias
    df_strategies = pd.DataFrame(columns=['market', 'timeframe', 'system_type', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2'])

    for  timeframe in timeframes: 
        print(market, timeframe)
        df_data = get_ohlcv_data(market, timeframe, init_date, end_date)
        df_indicators = df_indicators_gen(df_data)
        df_indicators.to_pickle(v_path + 'trading_systems_' + market + '/'+ market +'_'+ timeframe +'_indicators__' + init_date + '--' + end_date + '.pkl')
        df_above_conditions, df_below_conditions = df_conditions_gen(df_indicators)
        df_conditions = pd.concat([df_above_conditions, df_below_conditions], axis=1)     # unificamos los df de condiciones
        df_conditions = df_conditions.loc[:,~df_conditions.columns.duplicated()]
        df_conditions.to_pickle(v_path + 'trading_systems_' + market + '/'+ market +'_'+ timeframe +'_conditions__' + init_date + '--' + end_date + '.pkl')
        df_strategies = df_strategies.append(df_strategies_gen(df_above_conditions, df_below_conditions, timeframe), ignore_index=True)

    return df_strategies


def main_2(market, point_value, timeframes, init_date, end_date, min_pf, min_n_trades, df_strategies):
    
    # 2. para cada estrategia generada, realizamos el backtest correspondiente
    df_ratios = pd.DataFrame(columns=['market',	'timeframe', 'system_type', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2', 'n_trades', 'profit_factor', 'bmo', 'net_profit'])

    for i in range(len(df_strategies)):
        # obtenemos la informacion de la estrategia
        market = df_strategies['market'][i]
        
        timeframe = df_strategies['timeframe'][i]
        system_type = df_strategies['system_type'][i]
        in_condition_1 = df_strategies['in_condition_1'][i]
        in_condition_2 = df_strategies['in_condition_2'][i]
        out_condition_1 = df_strategies['out_condition_1'][i]
        out_condition_2 = df_strategies['out_condition_2'][i]
        df_conditions = pd.read_pickle(v_path + 'trading_systems_' + market + '/'+ market +'_'+ timeframe +'_conditions__' + init_date + '--' + end_date + '.pkl')
        #print(market, timeframe, system_type, in_condition_1, in_condition_2, out_condition_1, out_condition_2)
        df_ratios = df_ratios.append(backtest(market, point_value, timeframe, system_type, in_condition_1, in_condition_2, out_condition_1, out_condition_2, df_conditions, min_n_trades))

    return df_ratios

In [10]:
# definimos un df para almacenar los valores de point_value para cada market que queremos tener en el portfolio
point_value_df = pd.DataFrame({'market': ['NQ', 'ES', 'YM', 'CL', 'NG', 'GC', 'VX', 'TY'], 'point_value': [20, 50, 5, 1000, 10000, 100, 1000, 1000]})
point_value_df

,market,point_value
0,NQ,20
1,ES,50
2,YM,5
3,CL,1000
4,NG,10000
5,GC,100
6,VX,1000
7,TY,1000


# Criterio de preseleccion de estrategias

In [11]:
# para cada estrategia se realizan 2 backtests: uno historico largo (4 años), y otro corto pero mas reciente (4 meses)
# para el backtest largo, se consideran las estrategias con un profit factor mayor o igual a 1.4 y un numero de trades mayor o igual a 480
# para el backtest corto, se consideran las estrategias con un profit factor mayor o igual a 1.6 y un numero de trades mayor o igual a 40

# se definen los timeframes: '5min', '10min', '15min'

In [12]:
# short backtest
market = 'CL'
point_value = point_value_df[point_value_df['market'] == market]['point_value'].values[0]
timeframes =  ['5min', '10min', '15min'] 
init_date = '2022-09-01'
end_date = '2023-01-01'
min_n_trades = 40
min_pf = 1.6

df_strategies = main_1(market, timeframes, init_date, end_date)
df_ratios = main_2(market, point_value, timeframes, init_date, end_date, min_pf, min_n_trades, df_strategies)

print('numero de estrategias generadas: ', len(df_strategies))
# eliminamos aquellas estrategias con profit factor menor que min_pf
df_ratios = df_ratios[df_ratios.profit_factor >= min_pf]
df_ratios.to_pickle(v_path + 'trading_systems_' + market + '/'+ market + '_ratios__' + init_date + '--' + end_date + '.pkl')
df_ratios

CL 5min
CL 10min
CL 15min
numero de estrategias generadas:  739206


,market,timeframe,system_type,in_condition_1,in_condition_2,out_condition_1,out_condition_2,n_trades,profit_factor,bmo,net_profit
0,CL,5min,trend_long,a_condition_1,a_condition_24,b_condition_12,b_condition_17,44,1.697446,80.681818,3550.0
0,CL,5min,trend_long,a_condition_1,a_condition_24,b_condition_2,b_condition_8,46,1.697727,66.739130,3070.0
0,CL,5min,trend_long,a_condition_23,a_condition_25,b_condition_3,b_condition_21,42,1.644090,115.476190,4850.0
0,CL,5min,trend_long,a_condition_1,a_condition_24,b_condition_7,b_condition_22,45,2.016129,98.000000,4410.0
0,CL,5min,trend_long,a_condition_1,a_condition_24,b_condition_7,b_condition_14,45,1.909890,92.000000,4140.0
...,...,...,...,...,...,...,...,...,...,...,...
0,CL,15min,trend_short,b_condition_12,b_condition_25,a_condition_2,a_condition_3,44,1.853982,219.318182,9650.0
0,CL,15min,trend_short,b_condition_1,b_condition_19,a_condition_7,a_condition_14,117,1.618677,135.897436,15900.0
0,CL,15min,trend_short,b_condition_19,b_condition_26,a_condition_20,a_condition_23,40,1.771360,320.500000,12820.0
0,CL,15min,trend_short,b_condition_12,b_condition_27,a_condition_2,a_condition_11,47,1.910811,215.106383,10110.0


In [13]:
# long backtest
market = 'CL'
point_value = point_value_df[point_value_df['market'] == market]['point_value'].values[0]
timeframes =  ['5min', '10min', '15min']
init_date = '2018-09-01'
end_date = '2022-09-01'
min_n_trades = 480
min_pf = 1.4

df_strategies = main_1(market, timeframes, init_date, end_date)
df_ratios = main_2(market, point_value, timeframes, init_date, end_date, min_pf, min_n_trades, df_strategies)

print('numero de estrategias generadas: ', len(df_strategies))
# eliminamos aquellas estrategias con profit factor menor que min_pf
df_ratios = df_ratios[df_ratios.profit_factor >= min_pf]
df_ratios.to_pickle(v_path + 'trading_systems_' + market + '/'+ market + '_ratios__' + init_date + '--' + end_date + '.pkl')
df_ratios

CL 5min
CL 10min
CL 15min
numero de estrategias generadas:  739206


,market,timeframe,system_type,in_condition_1,in_condition_2,out_condition_1,out_condition_2,n_trades,profit_factor,bmo,net_profit
0,CL,5min,trend_long,a_condition_1,a_condition_24,b_condition_12,b_condition_26,517,1.473511,83.326886,43080.0
0,CL,5min,trend_long,a_condition_1,a_condition_24,b_condition_11,b_condition_26,532,1.473605,82.800752,44050.0
0,CL,15min,trend_short,b_condition_14,b_condition_24,a_condition_12,a_condition_21,503,1.449169,88.628231,44580.0
0,CL,15min,trend_short,b_condition_9,b_condition_24,a_condition_11,a_condition_21,491,1.471846,99.327902,48770.0
0,CL,15min,trend_short,b_condition_19,b_condition_24,a_condition_12,a_condition_21,499,1.469002,92.024048,45920.0
0,CL,15min,trend_short,b_condition_8,b_condition_24,a_condition_11,a_condition_21,492,1.473878,99.552846,48980.0
0,CL,15min,trend_short,b_condition_14,b_condition_24,a_condition_10,a_condition_21,500,1.408560,90.300000,45150.0
0,CL,15min,trend_short,b_condition_16,b_condition_24,a_condition_11,a_condition_21,494,1.473308,99.068826,48940.0
0,CL,15min,trend_short,b_condition_8,b_condition_24,a_condition_12,a_condition_21,501,1.450484,89.161677,44670.0
0,CL,15min,trend_short,b_condition_17,b_condition_24,a_condition_11,a_condition_21,489,1.472029,99.734151,48770.0
